In [1]:
pip install pandas mysql-connector-python openpyxl


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [6]:
import pandas as pd
import mysql.connector

In [7]:
cell_phones_df = pd.read_csv("cell_phones_total.csv")
population_df = pd.read_csv("pop.csv")


In [5]:
cell_phones_df.rename(columns={'geo': 'Country', 'time': 'Year', 'cell_phones_total': 'Cell_Phone_Total'}, inplace=True)
population_df.rename(columns={'geo': 'Country', 'time': 'Year', 'population': 'Population'}, inplace=True)


In [27]:
import mysql.connector


conn = mysql.connector.connect(
    host="localhost",
    user="root",  
    password="priya@2004", 
    database="bootcamp" 
)
if conn.is_connected():
    print("✅ Connected to MySQL!")

cursor = conn.cursor()


✅ Connected to MySQL!


In [7]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS CellPhones (
        Country VARCHAR(100),
        Year INT,
        Cell_Phone_Total BIGINT,
        PRIMARY KEY (Country, Year)
    )
""")

In [28]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Population (
        Country VARCHAR(100),
        Year INT,
        Population VARCHAR(20),  # VARCHAR to handle values like "3.28M"
        PRIMARY KEY (Country, Year)
    )
""")

conn.commit()
print("Tables created successfully!")


Tables created successfully!


In [29]:
for _, row in cell_phones_df.iterrows():
    cell_phone_total = row['Cell_Phone_Total'] if pd.notna(row['Cell_Phone_Total']) else 0
    cursor.execute(
        "INSERT IGNORE INTO CellPhones (Country, Year, Cell_Phone_Total) VALUES (%s, %s, %s)",
        (row['Country'], row['Year'], cell_phone_total)
    )

for _, row in population_df.iterrows():
    cursor.execute(
        "INSERT IGNORE INTO Population (Country, Year, Population) VALUES (%s, %s, %s)",
        (row['Country'], row['Year'], row['Population'])
    )

conn.commit()
print("Data inserted successfully!")

Data inserted successfully!


In [30]:
cursor.execute("""
    UPDATE CellPhones 
    SET Cell_Phone_Total = 0 
    WHERE Cell_Phone_Total IS NULL;
""")
conn.commit()
print("Updated NULL values to 0.")


Updated NULL values to 0.


In [31]:
cursor.execute("SELECT MAX(Year) FROM CellPhones;")
latest_year = cursor.fetchone()[0]
print(f"Latest Year: {latest_year}")


Latest Year: 2022


In [32]:
cursor.execute(f"""
    SELECT Country, Cell_Phone_Total 
    FROM CellPhones 
    WHERE Year = {latest_year}
    ORDER BY Cell_Phone_Total DESC
    LIMIT 10;
""")
top_countries = cursor.fetchall()

print("\n🔝 Top 10 Countries by Mobile Phone Usage in", latest_year)
for country in top_countries:
    print(country)
cursor.execute("""
    SELECT Year, Cell_Phone_Total 
    FROM CellPhones 
    WHERE Country = 'India'
    ORDER BY Year ASC;
""")
india_growth = cursor.fetchall()

print("\n📈 Mobile Growth in India")
for year, phones in india_growth:
    print(f"{year}: {phones}")



🔝 Top 10 Countries by Mobile Phone Usage in 2022
('Hong Kong, China', 292)
('UAE', 212)
('Libya', 205)
('Montenegro', 203)
('Seychelles', 192)
('El Salvador', 182)
('Kuwait', 181)
('Thailand', 176)
('Macao, China', 175)
('Qatar', 174)

📈 Mobile Growth in India
1960: 0
1961: 0
1962: 0
1963: 0
1964: 0
1965: 0
1966: 0
1967: 0
1968: 0
1969: 0
1970: 0
1971: 0
1972: 0
1973: 0
1974: 0
1975: 0
1976: 0
1977: 0
1978: 0
1979: 0
1980: 0
1981: 0
1982: 0
1983: 0
1984: 0
1985: 0
1986: 0
1987: 0
1988: 0
1989: 0
1990: 0
1991: 0
1992: 0
1993: 0
1994: 0
1995: 0
1996: 0
1997: 0
1998: 0
1999: 0
2000: 0
2001: 1
2002: 1
2003: 3
2004: 5
2005: 8
2006: 14
2007: 20
2008: 29
2009: 43
2010: 61
2011: 71
2012: 68
2013: 69
2014: 72
2015: 76
2016: 84
2017: 86
2018: 86
2019: 83
2020: 83
2021: 82
2022: 81


In [34]:
df = pd.read_sql("SELECT * FROM CellPhones WHERE Year = 2022 ORDER BY Cell_Phone_Total DESC;", conn)
df.to_csv("top_mobile_usage_2022.csv", index=False)
print("Data exported successfully to top_mobile_usage_2022.csv")



Data exported successfully to top_mobile_usage_2022.csv


C:\Users\Jaya Priya\AppData\Local\Temp\ipykernel_11952\4027481611.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM CellPhones WHERE Year = 2022 ORDER BY Cell_Phone_Total DESC;", conn)


In [35]:
cursor.close()
conn.close()
print("MySQL connection closed.")

MySQL connection closed.
